In [1]:
# import packages
import numpy as np
from scipy.io import loadmat
import pandas as pd

from rpy2.robjects import r
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import matplotlib.pyplot as plt
import seaborn as sns

bf_package=importr('BayesFactor')

%matplotlib qt

['/Users/teichmanna2/anaconda3/envs/mne/lib/python3.9/site-packages/rpy2']

In [ ]:
# chance decoding
chance=0.5
## load data from matlab struct [such as cosmomvpa results file]
x = loadmat('../data_colour/ds_stacked_realcolour.mat',squeeze_me=True,struct_as_record=False)
data=x['ds_stacked_realcolour']
df=pd.DataFrame(data.samples)

## load data from csv file [this should be participants x timepoints accuracy matrix]
# df = pd.read_csv('../data_colour/res.csv',header=None)

# loop over timepoints, make decoding accuracy into effect size and convert to an r object
n_timepoints = df.shape[1]
df_norm = pd.DataFrame(np.empty_like(df))
for t in range(n_timepoints):
  df_norm[t]=[(i - chance) for i in df[t]]
 
with localconverter(ro.default_converter + pandas2ri.converter):
  r_data = ro.conversion.py2rpy(df_norm)

In [ ]:
# loop over timepoints
bf=[]
for t in range(n_timepoints):
    results=bf_package.ttestBF(x=r_data[t],mu=0,rscale='medium',nullInterval=[0.5,float('inf')])
    bf.append(np.asarray(r['as.vector'](results))[0])


In [ ]:
# plot as a quick peak
x = np.arange(-100,805,5)
y = bf
plt.stem(x, y,bottom=1,linefmt=None, markerfmt=None, basefmt=None)
plt.yscale('log')
ax = plt.gca()
ax.set_ylim(bottom=10**-10, top=10**10)

In [ ]:
# plot BFs with coloured stems (like the matlab version)
plt.close('all')
fig, ax = plt.subplots(figsize=(7,3))

bf_cols = sns.color_palette("coolwarm", 500)
exponential_minmax = 10
val_col_map = np.logspace(-exponential_minmax,exponential_minmax,num=500)

x = np.arange(-100,805,5)
y = bf
markerline, stemlines, baseline = ax.stem(x, y,bottom=1,linefmt='k', markerfmt=None, basefmt=None)

markerline.set_markerfacecolor('w')
markerline.set_markeredgecolor('w')
baseline.set_color('k')
stemlines.set_linewidth(0.5)

cols_idx = [np.argmin(np.abs(val_col_map-i)) for i in y]  
[ax.plot(x[i],y[i],color=bf_cols[cols_idx[i]],marker='.',markersize=10,lw=0,markeredgecolor=None) for i in range(len(cols_idx))]
ax.set_yscale('log')
ax.set_ylim([10**-exponential_minmax,10**exponential_minmax])
ax.set_yticks([ 1.e-10, 1, 1.e+10])
ax.get_xaxis().get_major_formatter().labelOnlyBase = True

ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

ax.set_xlabel('time (s)',fontsize=14)
ax.set_ylabel('Bayes Factor\n(log)',fontsize=14)

plt.tight_layout()